<a href="https://colab.research.google.com/github/rohini-rs1/TSF1/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import datetime
import csv
import tweepy
import numpy as np
import pandas as pd


consumer_key = 'uUtNj3h19KiTvsNJUMDGVPtZR'
consumer_secret = '9OQPHzxL49qmipveLqMmBco3dyJK4IasjIeM1Efay6OvgIehZq'
access_token = '1201035700499410944-wLJ7MIKlTWjSvDvhGOqYUFRzj5h2MX'
access_token_secret = 'NRTbhxmkZHbzbayVIBv1i3vVdPVFbas8jnu5sxyKolU3l'

In [2]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
for tweet in api.search('AmazonIN'):
    print(tweet.text)

RT @jin_dionysus: Hey @amazonIN  is there any chance of making @BTS_twt 's official albums available in india
RT @jin_dionysus: Hey @amazonIN  is there any chance of making @BTS_twt 's official albums available in india
@zingoy_cashback @Contestmumbai @Giveaway_Contst @Contest_in @amazonIN @Contest_Hub Please select fastest answers will be winners
RT @jin_dionysus: Hey @amazonIN  is there any chance of making @BTS_twt 's official albums available in india
@itsJustice4SSR @KanganaTeam @amazon @amazonIN @amazonmusic Really what did they do now i mean kuch bhi twitter hai… https://t.co/WhKBYVwHdd
RT @jin_dionysus: Hey @amazonIN  is there any chance of making @BTS_twt 's official albums available in india
@zingoy_cashback @Contestmumbai @Giveaway_Contst @Contest_in @amazonIN @Contest_Hub Ready 
Very exited
RT @jin_dionysus: Hey @amazonIN  is there any chance of making @BTS_twt 's official albums available in india
RT @jin_dionysus: Hey @amazonIN  is there any chance of making @BTS_twt 's o

In [4]:
df = pd.DataFrame(columns = ['Tweets', 'User', 'User_statuses_count', 
                             'user_followers', 'User_location', 'rt_count', 'tweet_date'])

In [5]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i+=1
        if i == 2000:
            break
        else:
            pass

In [ ]:
stream(data = ['AmazonIN'], file_name = 'AmazonIN')

In [ ]:
df.head()

In [ ]:
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl', offline=True)

In [ ]:
df.info()

In [ ]:
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [ ]:
def analyze_emotions(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity <= -0.75:
        return 'Anger'
    elif analysis.sentiment.polarity <= -0.50:
        return 'disgust'
    elif analysis.sentiment.polarity <= -0.25:
        return 'Anticipation'
    elif analysis.sentiment.polarity <= -0.01:
        return 'fear'
    elif analysis.sentiment.polarity == 0:
        return 'joy'
    elif analysis.sentiment.polarity <= 0.25:
        return 'sadness'
    elif analysis.sentiment.polarity <= 0.50:
        return 'surprise'
    elif analysis.sentiment.polarity <= 0.75:
        return 'trust'
    else:
        return 'All Good'

In [ ]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity < 0:
        return 'Negative'
    else:
        return 'Neutral' 

In [ ]:
import re
df['clean_tweet'] = df['Tweets'].apply(lambda x: clean_tweet(x))
df['Sentiment']=df['clean_tweet'].apply(lambda x: analyze_sentiment(x))
df['Emotions'] = df['clean_tweet'].apply(lambda x: analyze_emotions(x))
df['sentimentpolarity'] = df.clean_tweet.apply(lambda x: TextBlob(str(x)).sentiment.polarity)

In [ ]:
df

In [ ]:
df.drop_duplicates(subset='clean_tweet',inplace=True)

In [ ]:
df

In [ ]:
df['Sentiment'].value_counts().iplot(kind='bar', xTitle='Emotions',
                                    yTitle='Count', title='Overall Sentiment Distribution')

In [ ]:
df['Emotions'].value_counts().iplot(kind='bar', xTitle='Emotions',
                                    yTitle='Count', title='Emotionswise Sentiment Distribution')

In [ ]:
all_tweets = ' '.join(tweet for tweet in df['Emotions'])


wordcloud = WordCloud(stopwords=STOPWORDS).generate(all_tweets)

In [ ]:
df_freq = pd.DataFrame.from_dict(data = wordcloud.words_, orient='index')
df_freq = df_freq.head(20)
df_freq.plot.bar()